In [85]:
#allow autoreload modules at every execution if requested
%load_ext autoreload

import tempfile
import unittest

from pathlib import Path

import numpy as np
import pandas as pd

from cs_parser import CsParser, mappings
from star_parser import StarTabDf, StarParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
%autoreload 2
test_file = (
        "/mnt/DATA/andrea/20210226_NeCen_BRCA1A_Ub/relion4/P52_J977_passthrough_particles.cs"
    )
parser = CsParser(test_file)
array = parser.parse_array()
# array.rename_columns()

In [87]:
from pprint import pprint
a = array.cs_array.copy()
dtypes = a.dtype.fields
problematic = []
clean = []
for index, value in enumerate(list(a[0])):
    if isinstance(value, np.ndarray):
        key = list(a.dtype.fields.keys())[index]
        problematic.append(key)
    else:
        key = list(a.dtype.fields.keys())[index]
        clean.append(key)
clean[:]

['uid',
 'blob/path',
 'blob/idx',
 'blob/psize_A',
 'blob/sign',
 'blob/import_sig',
 'location/micrograph_uid',
 'location/exp_group_id',
 'location/micrograph_path',
 'location/center_x_frac',
 'location/center_y_frac',
 'alignments2D/split',
 'alignments2D/pose',
 'alignments2D/psize_A',
 'alignments2D/error',
 'alignments2D/error_min',
 'alignments2D/resid_pow',
 'alignments2D/slice_pow',
 'alignments2D/image_pow',
 'alignments2D/cross_cor',
 'alignments2D/alpha',
 'alignments2D/alpha_min',
 'alignments2D/weight',
 'alignments2D/pose_ess',
 'alignments2D/shift_ess',
 'alignments2D/class_posterior',
 'alignments2D/class',
 'alignments2D/class_ess',
 'alignments_class_1/split',
 'alignments_class_1/psize_A',
 'alignments_class_1/error',
 'alignments_class_1/error_min',
 'alignments_class_1/resid_pow',
 'alignments_class_1/slice_pow',
 'alignments_class_1/image_pow',
 'alignments_class_1/cross_cor',
 'alignments_class_1/alpha',
 'alignments_class_1/alpha_min',
 'alignments_class_1/we

In [88]:
mapping_coords = {0:'x', 1:'y', 2:'z'}
mapping_angles = {0:'alpha', 1:'beta', 2:'gamma'}
rascals = a[problematic]
index_column = range(a.shape[0])
reformed = pd.DataFrame(index_column)
# make dummy zero array column
# reformed = np.zeros((rascals[problematic[0]].shape[0], 1))
for key in problematic:
    #if the name of the column is poses, they are angles;
    #otherwise they are shifts
    if 'pose' in key:
        mapping = mapping_angles
    else:
        mapping = mapping_coords
    #split the array into a list of n single columns
    split_arrays = np.hsplit(rascals[key], rascals[key].shape[-1])
    #rename their index
    for index, array in enumerate(split_arrays):
        new_dtype = (f'{key}_{mapping[index]}')
        reformed[new_dtype] = pd.DataFrame(array)
reformed

,0,blob/shape_x,blob/shape_y,location/micrograph_shape_x,location/micrograph_shape_y,alignments2D/shift_x,alignments2D/shift_y,alignments_class_1/shift_x,alignments_class_1/shift_y,alignments_class_1/pose_alpha,alignments_class_1/pose_beta,alignments_class_1/pose_gamma,alignments_class_0/shift_x,alignments_class_0/shift_y,alignments_class_0/pose_alpha,alignments_class_0/pose_beta,alignments_class_0/pose_gamma
0,0,420,420,4092,5760,-1.332032,-1.332032,-0.1625,-2.4375,0.981748,1.318347,0.532949,-3.0875,2.1125,0.476849,-0.196350,0.701248
1,1,420,420,4092,5760,-2.066406,2.136719,1.1375,-4.0625,-2.103745,1.150047,2.328145,-3.0875,4.3875,-0.532949,-0.196350,-2.720844
2,2,420,420,4092,5760,-5.597656,16.863281,-1.1375,0.1625,1.374447,1.374447,0.813448,-2.4375,0.1625,-1.879346,2.889143,-1.430547
3,3,420,420,4092,5760,-14.597656,-25.597658,-1.1375,-0.1625,-1.206147,-0.420749,1.374447,0.1625,-5.0375,-2.720844,1.654946,-1.206147
4,4,420,420,4092,5760,0.730470,5.332032,4.0625,-0.8125,1.206147,-1.150047,-1.374447,2.7625,3.0875,-2.384244,0.420749,0.813448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113412,113412,420,420,4092,5760,-55.660156,-21.066408,-5.6875,-2.7625,0.140250,2.720844,0.364649,-6.6625,-0.1625,2.159845,-1.823246,0.532949
113413,113413,420,420,4092,5760,-12.324219,-1.605469,-3.0750,1.7250,-1.949632,-0.175250,-2.371995,-0.9750,1.5750,-0.523065,1.759765,0.358525
113414,113414,420,420,4092,5760,35.066406,-16.464844,-3.8250,3.8250,-1.016141,-1.285393,-0.266622,-0.0750,-6.3750,-0.714968,-1.677946,2.389789
113415,113415,420,420,4092,5760,16.464844,-7.128909,0.5250,-5.0250,1.325368,-0.627129,-0.974620,0.5250,-6.3750,0.236792,-2.095926,2.020194


In [89]:
good = a[clean]
df = pd.DataFrame(good)
new_columns = []
for c in df.columns:
    if c in mappings and mappings[c]:
        new_columns.append(mappings[c])
    else:
        new_columns.append(c)
df.columns = new_columns

In [90]:
swapxy = True
inverty = True
invertx = True
frac_coords = df[['FracCoordinateX','FracCoordinateY']]
mic_size = reformed[['location/micrograph_shape_x','location/micrograph_shape_y']]
abs_coords = pd.DataFrame()
#this version of pandas complains about chained assignment even though it does not have a chained assignment
pd.set_option('mode.chained_assignment', None)
if invertx:
    frac_coords.loc[:, 'FracCoordinateX'] = 1 -frac_coords.loc[:,'FracCoordinateX']
if inverty:
    frac_coords.loc[:, 'FracCoordinateY'] = 1- frac_coords.loc[:,'FracCoordinateY']
pd.set_option('mode.chained_assignment', 'warn')
if swapxy:
    abs_coords['CoordinateX'] = pd.DataFrame(frac_coords['FracCoordinateX'] * reformed['location/micrograph_shape_y'])
    abs_coords['CoordinateY'] = pd.DataFrame(frac_coords['FracCoordinateY'] * reformed['location/micrograph_shape_x'])
    abs_coords = abs_coords.applymap(np.rint).astype(int)
else:
    abs_coords['CoordinateX'] = pd.DataFrame(frac_coords['FracCoordinateX'] * reformed['location/micrograph_shape_x'])
    abs_coords['CoordinateY'] = pd.DataFrame(frac_coords['FracCoordinateY'] * reformed['location/micrograph_shape_y'])
    abs_coords = abs_coords.applymap(np.rint).astype(int)
abs_coords


,CoordinateX,CoordinateY
0,5445,3773
1,443,3255
2,1475,2925
3,5522,2802
4,3285,2500
...,...,...
113412,3894,1896
113413,483,252
113414,2232,616
113415,4822,1209


In [91]:
df[abs_coords.columns] = abs_coords
for column in ['ImageName', 'MicrographName']:
    df[column] = df[column].str.decode('utf-8')

"""
        df[star.Relion.COORDX] = cs[u'location/center_x_frac']
        df[star.Relion.COORDY] = cs[u'location/center_y_frac']
        if invertx:
            df[star.Relion.COORDX] = 1 - df[star.Relion.COORDX]
        if inverty:
            df[star.Relion.COORDY] = 1 - df[star.Relion.COORDY]
        if swapxy:
            df[star.Relion.COORDS] = np.round(df[star.Relion.COORDS] *
                                              cs['location/micrograph_shape'][:, ::-1]).astype(np.int)
        else:
            df[star.Relion.COORDS] = np.round(df[star.Relion.COORDS] * cs['location/micrograph_shape']).astype(np.int)
"""

In [92]:
%autoreload 2
df = df.astype(str)
starparser = StarParser('/mnt/DATA/andrea/20210226_NeCen_BRCA1A_Ub/relion4/P52_J977_passthrough_particles.star', create=True)
data = starparser.read_df(df)

In [93]:
tab = data['data_']
tab.keep_only_columns(['MicrographName','CoordinateX','CoordinateY'], store=True)
tab.trim_column_values('MicrographName', len("J113/motioncorrected/012824420616707970296_"), store=True)


,MicrographName,CoordinateX,CoordinateY
0,FoilHole_26045258_Data_26043371_26043373_20210...,5445,3773
1,FoilHole_26045258_Data_26043371_26043373_20210...,443,3255
2,FoilHole_26045258_Data_26043371_26043373_20210...,1475,2925
3,FoilHole_26045258_Data_26043371_26043373_20210...,5522,2802
4,FoilHole_26045258_Data_26043371_26043373_20210...,3285,2500
...,...,...,...
113412,FoilHole_26052136_Data_26045787_26045789_20210...,3894,1896
113413,FoilHole_26052136_Data_26045787_26045789_20210...,483,252
113414,FoilHole_26052136_Data_26045787_26045789_20210...,2232,616
113415,FoilHole_26052136_Data_26045787_26045789_20210...,4822,1209


In [94]:
tab.remove_prefix_from_column('_patch_aligned_doseweighted', 'MicrographName', store=True)["MicrographName"][0]
# tab.remove_prefix_from_column("'", 'MicrographName', store=True)["MicrographName"][0]

'FoilHole_26045258_Data_26043371_26043373_20210226_112933_fractions.mrc'

In [95]:
starparser.write_out(to_file=True)

Data written to /mnt/DATA/andrea/20210226_NeCen_BRCA1A_Ub/relion4/P52_J977_passthrough_particles.star


In [96]:
split = tab.to_df().groupby('MicrographName')
work_folder = Path('/mnt/DATA/andrea/20210226_NeCen_BRCA1A_Ub/relion4')
split_dir = work_folder/'P53_J977_clean_starfiles_test_swap_invx_invy'
if not split_dir.exists():
    split_dir.mkdir()
for fname, data in split:
    x = StarTabDf(data)
    fname = fname.replace('.mrc','.star')
    with open(split_dir/fname, 'w') as f:
        f.write(x.to_star())